In [1]:
import dash
from dash import dcc, html, Input, Output, State, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
from sqlalchemy import create_engine
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
import numpy as np
from jupyter_dash import JupyterDash
import pdfkit
import base64
from io import BytesIO
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import pyodbc
from urllib.parse import quote_plus
import io
import oracledb
from datetime import datetime
from dash.exceptions import PreventUpdate
from concurrent.futures import ThreadPoolExecutor
import os

In [4]:
# Parámetros de conexión
server = '10.0.0.133'
database = 'DB_GENERAL'
database2= 'DB_TRAMITE_DOCUMENTARIO'  # Reemplaza con el nombre de tu base de datos
username = 'sa'
password = 'Essalud23**'
def create_connection():
    conn = None
    try:
        # Establecer conexión
        conn = pyodbc.connect(r'DRIVER={ODBC Driver 18 for SQL Server};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'UID={username};'
        f'PWD={password};'
        'TrustServerCertificate=yes;')
        print("Conexión exitosa.")
    except Exception as e:
        print(f"Error al conectar: {e}")
    return conn

In [17]:
var="SG"

In [36]:
def fetch_data(var):
    conn = create_connection()
    if conn is None:
        raise ConnectionError("No se pudo establecer la conexión a la base de datos.")
    try:
        query = f"""
SELECT 
    d.hoja_tramite,
    d.tipo_hoja,
    cdi.DESCRIPCION,
    d.INDICATIVO_OFICIO,
    p.RAZON_SOCIAL,
    d.FECHA_RECEPCION,
    d.ASUNTO,
    md.AUDIT_REC,
    do.SIGLAS AS SIGLAS_ORIGEN,
    do.DEPENDENCIA AS DEPENDENCIA_ORIGEN,
    dd.SIGLAS AS SIGLAS_DESTINO,
    dd.DEPENDENCIA AS DEPENDENCIA_DESTINO,
    ht.APELLIDOS_TRABAJADOR,
    ht.NOMBRES_TRABAJADOR,
    ed.ID_ESTADO_DOCUMENTO,
    ed.DESCRIPCION AS ESTADO_DOCUMENTO,
    md.AUDIT_TRAB_DER AS FECHA_DELEGADO,
    md.derivado,
    md.finalizado,

    -- Columna DIFERENCIA_DIAS
    DATEDIFF(DAY, md.AUDIT_TRAB_DER, GETDATE()) AS DIFERENCIA_DIAS,

    -- Columna ESTADO
    CASE
        WHEN ed.ID_ESTADO_DOCUMENTO <> 2 
          AND ed.ID_ESTADO_DOCUMENTO <> 10 
          AND md.derivado = 0 
          AND md.finalizado = 0 THEN 'Pendiente'
        ELSE 'Otros'
    END AS ESTADO

FROM 
    DB_TRAMITE_DOCUMENTARIO.web_tramite.MOVIMIENTO_DOCUMENTO md
LEFT OUTER JOIN 
    DB_TRAMITE_DOCUMENTARIO.web_tramite.DOCUMENTO d ON md.ID_DOCUMENTO = d.ID_DOCUMENTO
LEFT OUTER JOIN 
    DB_GENERAL.dbo.PERSONA p ON d.ID_PERSONA = p.ID
LEFT OUTER JOIN 
    DB_TRAMITE_DOCUMENTARIO.dbo.CLASE_DOCUMENTO_INTERNO cdi ON d.ID_CLASE_DOCUMENTO_INTERNO = cdi.ID_CLASE_DOCUMENTO_INTERNO
LEFT OUTER JOIN 
    DB_TRAMITE_DOCUMENTARIO.dbo.ESTADO_DOCUMENTO ed ON ed.ID_ESTADO_DOCUMENTO = d.ID_ESTADO_DOCUMENTO
LEFT OUTER JOIN 
    DB_GENERAL.jcardenas.H_DEPENDENCIA do ON do.CODIGO_DEPENDENCIA = md.ID_DEPENDENCIA_ORIGEN 
LEFT OUTER JOIN 
    DB_GENERAL.jcardenas.H_DEPENDENCIA dd ON dd.CODIGO_DEPENDENCIA = md.ID_DEPENDENCIA_DESTINO
LEFT JOIN 
    DB_GENERAL.jcardenas.H_TRABAJADOR ht ON ht.CODIGO_TRABAJADOR = md.codigo_trabajador
WHERE
    (do.SIGLAS LIKE '{var}' OR dd.SIGLAS LIKE '{var}')
    AND
    md.AUDIT_TRAB_DER = (
        SELECT MAX(sub_md.AUDIT_TRAB_DER)
        FROM DB_TRAMITE_DOCUMENTARIO.web_tramite.MOVIMIENTO_DOCUMENTO sub_md
        LEFT OUTER JOIN DB_TRAMITE_DOCUMENTARIO.web_tramite.DOCUMENTO sub_d ON sub_md.ID_DOCUMENTO = sub_d.ID_DOCUMENTO
        WHERE sub_d.hoja_tramite = d.hoja_tramite
    )
        """
        result = pd.read_sql(query, conn)
        result["Hoja de trámite"]=result["hoja_tramite"]+"-"+result["tipo_hoja"]


    finally:
        conn.close()  # Cerrar la conexión al final
    return result

In [46]:
data=fetch_data(var)
data.columns

Conexión exitosa.


C:\Users\kings\AppData\Local\Temp\ipykernel_16924\3811788539.py:66: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


Index(['hoja_tramite', 'tipo_hoja', 'DESCRIPCION', 'INDICATIVO_OFICIO',
       'RAZON_SOCIAL', 'FECHA_RECEPCION', 'ASUNTO', 'AUDIT_REC',
       'SIGLAS_ORIGEN', 'DEPENDENCIA_ORIGEN', 'SIGLAS_DESTINO',
       'DEPENDENCIA_DESTINO', 'APELLIDOS_TRABAJADOR', 'NOMBRES_TRABAJADOR',
       'ID_ESTADO_DOCUMENTO', 'ESTADO_DOCUMENTO', 'FECHA_DELEGADO', 'derivado',
       'finalizado', 'DIFERENCIA_DIAS', 'ESTADO', 'Hoja de trámite'],
      dtype='object')

In [81]:
tipdoc= data["tipo_hoja"].unique()
options_razon_social=data["RAZON_SOCIAL"].dropna().unique()
options_razon_social

array(['FONDO NACIONAL DE FINANCIAMIENTO DE LA A CTIVIDAD EMPRESARIAL DEL ESTADO - FONAFE O FONAFE',
       'MINISTERIO DE CULTURA',
       'MINISTERIO DE TRANSPORTES Y COMUNICACIONES',
       'MINISTERIO DE SALUD', 'CONTRALORIA GENERAL DE LA REPUBLICA',
       'SEGURO SOCIAL DE SALUD',
       'ORGANISMO DE EVALUACION Y FISCALIZACION AMBIENTAL - OEFA',
       'BOTTGER BRAVO GEORGINA BELEN', 'RAMOS CASTILLO BRAYAN MARTIN',
       'PAZ MEDINA JORGE ARTURO', 'SANCHEZ FERNANDEZ WALTER WILFREDO',
       'SANABRIA LOPEZ JOSE MIGUEL', 'ELITE EXPRESS COURIER S.A.C.',
       'PEREYRA VILLANUEVA IVAN', 'CONGRESO DE LA REPUBLICA',
       'MINIST.DE TRABAJO Y PROMOCION DEL EMPLEO',
       'DROGUERIA SINMA S.A.C.', 'VALENCIA ANCI MARCO ANTONIO',
       'BASE HOSPITAL II CHOCOPE', 'VIPROSEG',
       'CABRERA MARCHAN MARTIN RAUL', 'ORDOÑEZ MENDOZA ALEX CRISTHOFER',
       'MORENO LUNA HAROLD',
       'sindicato nacional de profesionales administrativos del seguro social de salud - essalud -sinaproa',

In [54]:
def get_unique_options():
    connection = create_connection()
    if connection:
        try:
            cursor = connection.cursor()
            query = """SELECT DEPENDENCIA
                       FROM DB_GENERAL.jcardenas.H_DEPENDENCIA"""
            cursor.execute(query)
            # Retornar solo los valores de la columna 'DEPENDENCIA'
            options = [row[0] for row in cursor.fetchall()]
            cursor.close()
            return options
        finally:
            connection.close()
    return []

# Llamada a la función para obtener los valores de la columna 'DEPENDENCIA'
options = get_unique_options()

Conexión exitosa.


In [100]:
external_stylesheets = [
    dbc.themes.BOOTSTRAP,
    'https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css',  # Bootstrap CSS
    'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.1/css/all.min.css'  # Bootstrap Icons
]

# Crear la aplicación Dash
app = dash.Dash(__name__, suppress_callback_exceptions=True, external_stylesheets=external_stylesheets)

app.layout = dbc.Container([
    
    # Título
dbc.Row([
    # Título y logotipo en la misma fila
    dbc.Col([
        html.Div(style={'height': '12px'}),
        html.H1("Seguimiento de trámites pendientes", style={'color': '#0064AF', 'fontSize': '28px', 'textAlign': 'Left', 'font-weight': 'bold', 'fontFamily': 'Calibri'}),
        html.H2("Fuente: SGD Sede Central", style={'color': '#0064AF', 'fontSize': '12px'}),
    ], width=10),  # Ajusta el ancho según sea necesario
    dbc.Col([
        html.Img(src="/assets/Logotipo sin Slogan_Horizontal_Color Transparente.png", alt="Essalud", width="150"),
    ], width=2, className="d-none d-lg-flex align-items-center justify-content-end"),
]),
    html.Hr(style={'border': '2px solid #0064AF'}),
    dbc.Row([
        # Date Picker for Start Date
        dbc.Col([
            html.Label("Hoja de trámite", style={'font-size': '16px', 'color': '#0064AF'}),
            dcc.Input(id='HT',style={'font-size': '12px', 'height': '40px'}),
                ], width=9, md=3, lg=1),

        dbc.Col([
            html.Label("Tipo de documento", style={'font-size': '16px', 'color': '#0064AF'}),
            dcc.Dropdown(
                id='tipdoc',
                options=tipdoc,
                optionHeight=35,
                style={'height': '41px', 'width': '100%'}
        ),],width=9, md=3, lg=2),

        dbc.Col([
            html.Label("Razón social", style={'font-size': '16px', 'color': '#0064AF'}),
            dcc.Dropdown(
                id='razsoc',
                options=options_razon_social,
                optionHeight=60,
                style={'height': '41px', 'width': '100%', 'font-size': '16px'}
        ),],width=9, md=3, lg=3),

        dbc.Col([
            html.Label("Fecha delegado", style={'font-size': '16px', 'color': '#0064AF'}),
            dcc.DatePickerRange(
                id='date-picker-range',
                start_date_placeholder_text="Fecha inicio",
                end_date_placeholder_text="Fecha fin",
                display_format='DD-MM-YYYY',
                style={'height': '100px','font-size': '13px' },),
                
            html.Div(id='output-container')],width=9, md=3, lg=2),
            dbc.Col(
            
        ),

    ], className="mb-4"),

], fluid=True)






if __name__ == '__main__':
    app.run_server(debug=True)
